### SQL Homework
Use this notebook to answer the questions.
It can be in the same project as the previous homework.  
When you are ready, **upload** to your github repo, and send me the link (just zip a txt file with the repo's address, and upload it as the homework).  
If your repo is private, invite me: balazs.balogh@cubixedu.com.

#### Import the SparkSession, create it then load the taxi data (yellow_tripdata_2024-08.parquet)

In [1]:
from datetime import datetime, date

from pyspark.sql import SparkSession
from pyspark.sql.types import (
    StructType,
    StructField,
    IntegerType,
    StringType,
    DateType,
    TimestampType
)

spark = (
    SparkSession
    .builder
    .appName("SQL")
    .master("local[*]")
    .getOrCreate()
)

taxi_data_2024_08 = (
    spark
    .read
    .format("parquet")
    .load("C:\\Users\\SZABGAB\\cubix_data_engineer_pyspark_tutorial\\src\\data\\yellow_tripdata_2024-08.parquet")
)

taxi_data_2024_08.createOrReplaceTempView("taxi_2024_08")

#### 1. What is the total fare amount for all trips?  
Please round the answer to two decimal places.

In [ ]:
spark.sql("""
    
    SELECT
        ROUND(SUM(fare_amount),2) AS total_fare_amount
    FROM
        taxi_2024_08

""").show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1| 2024-08-01 00:21:00|  2024-08-01 00:36:13|              1|          7.4|         1|                 N|         138|          80|           1|       28.9| 7.75|    0.5|      7.6

#### 2. Show the maximum fare amount, minimum fare amount, and average fare amount for each payment type. Order by payment type.
Round where you need to two decimal places.

In [12]:
spark.sql("""
    
    SELECT
        payment_type,
        ROUND(MIN(fare_amount),2) AS min_fare_amount,
        ROUND(MAX(fare_amount),2) AS max_fare_amount,
        ROUND(AVG(fare_amount),2) AS avg_fare_amount
    FROM
        taxi_2024_08
    GROUP BY
        payment_type

""").show()

+------------+---------------+---------------+---------------+
|payment_type|min_fare_amount|max_fare_amount|avg_fare_amount|
+------------+---------------+---------------+---------------+
|           1|         -108.7|          650.0|           20.6|
|           3|         -999.0|          999.0|           6.26|
|           2|        -1174.1|         1386.2|          19.09|
|           4|         -900.0|          900.0|           1.37|
|           0|          -72.2|         394.76|          19.62|
+------------+---------------+---------------+---------------+



#### 3. For trips with a fare amount greater than 20, what is the total tip amount for each day (based on the tpep_pickup_datetime)?
Round the tip to two decimal places, and order the results from highest total tip amount.  
Hint: Check DATE() function, to convert tpep_pickup_datetime to date, to get only the YYYY-MM-DD.

In [18]:
spark.sql("""
    
    SELECT
        TO_DATE(tpep_pickup_datetime, 'yyyy/MM/dd') AS pickup_date,
        ROUND(SUM(tip_amount),2) AS total_tip_amount
    FROM
        taxi_2024_08
    WHERE
        fare_amount > 20
    GROUP BY
        TO_DATE(tpep_pickup_datetime, 'yyyy/MM/dd')
    ORDER BY
          1 

""").show()

+-----------+----------------+
|pickup_date|total_tip_amount|
+-----------+----------------+
| 2024-08-01|       176890.45|
| 2024-08-02|       169422.85|
| 2024-08-03|        163030.6|
| 2024-08-04|       170867.65|
| 2024-08-05|       179211.92|
| 2024-08-06|       177281.03|
| 2024-08-07|       194520.05|
| 2024-08-08|       191036.86|
| 2024-08-09|       156713.23|
| 2024-08-10|        149603.8|
| 2024-08-11|       174783.43|
| 2024-08-12|       156581.92|
| 2024-08-13|       152027.62|
| 2024-08-14|       164499.13|
| 2024-08-15|       173499.16|
| 2024-08-16|       161987.27|
| 2024-08-17|       149333.67|
| 2024-08-18|       155993.56|
| 2024-08-19|       181302.43|
| 2024-08-20|       174724.38|
+-----------+----------------+
only showing top 20 rows



#### 4. For each trip, show the fare amount along with a column that indicates if the trip was "expensive" (greater than 30) or "cheap" (less than or equal to 30).
Hint: Use CASE WHEN for deciding on expensive, or cheap.

In [21]:
spark.sql("""
    
    SELECT
        tpep_pickup_datetime,
        CASE
          WHEN fare_amount > 30 THEN "expensive"
          ELSE "cheap"
        END fare_category
    FROM
        taxi_2024_08
    LIMIT 10


""").show()

+--------------------+-------------+
|tpep_pickup_datetime|fare_category|
+--------------------+-------------+
| 2024-08-01 00:21:00|        cheap|
| 2024-08-01 00:20:01|    expensive|
| 2024-08-01 00:17:52|    expensive|
| 2024-08-01 00:49:08|        cheap|
| 2024-08-01 00:38:52|        cheap|
| 2024-08-01 00:57:59|        cheap|
| 2024-08-01 00:15:46|        cheap|
| 2024-08-01 00:32:17|        cheap|
| 2024-08-01 00:48:16|        cheap|
| 2024-07-31 23:30:15|        cheap|
+--------------------+-------------+



#### 5. Find the first trip (based on tpep_pickup_datetime) for each VendorID and display the fare amount.
Hint: You can use CTE with ROW_NUMBER().

In [24]:
spark.sql("""
    
    SELECT
        VendorID,
        fare_amount as first_fare_amount
    FROM
        (
        SELECT
            VendorID,
            fare_amount,
            ROW_NUMBER() OVER (PARTITION BY VendorID ORDER BY tpep_pickup_datetime) RN
        FROM
            taxi_2024_08
        )
    WHERE
        RN = 1

""").show()

+--------+-----------------+
|VendorID|first_fare_amount|
+--------+-----------------+
|       1|             70.0|
|       2|             10.7|
+--------+-----------------+



#### 7. Calculate the average trip distance for each VendorID, and assign a label of 'Above Average' or 'Below Average' for each trip based on the distance relative to the VendorID’s average trip distance.
Hint: CTE joined back to the main DataFrame.

In [28]:
spark.sql("""
    
    WITH avg_trip_dist_by_vendor AS
        (
        SELECT
            VendorID,
            AVG(TRIP_DISTANCE) OVER (PARTITION BY VendorID) AS average_trip_distance            
        FROM
            taxi_2024_08
        )
    SELECT
        t.VendorID,
        t.tpep_pickup_datetime,
        t.trip_distance,
        a.average_trip_distance,        
        CASE
            WHEN t.trip_distance >= a.average_trip_distance THEN "Above Average"
            ELSE "Below Average"
        END AS fare_category        
    FROM
        taxi_2024_08 t
    INNER JOIN
        avg_trip_dist_by_vendor a
    ON
        t.VendorID = a.VendorID
        

""").show(10)

+--------+--------------------+-------------+---------------------+-------------+
|VendorID|tpep_pickup_datetime|trip_distance|average_trip_distance|fare_category|
+--------+--------------------+-------------+---------------------+-------------+
|       1| 2024-08-31 23:24:04|          1.9|    4.034096101093453|Below Average|
|       1| 2024-08-31 23:53:21|          3.0|    4.034096101093453|Below Average|
|       1| 2024-08-31 23:09:34|         11.2|    4.034096101093453|Above Average|
|       1| 2024-08-31 23:19:32|          9.1|    4.034096101093453|Above Average|
|       1| 2024-08-31 23:32:03|          1.9|    4.034096101093453|Below Average|
|       1| 2024-08-31 23:14:45|          0.0|    4.034096101093453|Below Average|
|       1| 2024-08-31 23:39:25|         11.2|    4.034096101093453|Above Average|
|       1| 2024-08-31 23:27:49|          2.2|    4.034096101093453|Below Average|
|       1| 2024-08-31 23:28:04|          1.5|    4.034096101093453|Below Average|
|       1| 2024-